In [1]:
#data source: https://www.kaggle.com/datasets/hernan4444/anime-recommendation-database-2020

In [2]:
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
from lightfm import cross_validation

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix

import itertools
import re

from sklearn import preprocessing

from tqdm import tqdm

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
anime_s = pd.read_csv('/home/dmitry-ds/rec-sys/data/anime_with_synopsis.csv')

In [5]:
anime_s.head()

,MAL_ID,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...


In [6]:
anime_s.shape

(16214, 5)

In [7]:
anime = pd.read_csv('/home/dmitry-ds/rec-sys/data/anime.csv')

In [8]:
anime.head()

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,Producers,Licensors,Studios,Source,Duration,Rating,Ranked,Popularity,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,Original,24 min. per ep.,R - 17+ (violence & profanity),28.0,39,1251960,61971,105808,718161,71513,26678,329800,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,"Sunrise, Bandai Visual",Sony Pictures Entertainment,Bones,Original,1 hr. 55 min.,R - 17+ (violence & profanity),159.0,518,273145,1174,4143,208333,1935,770,57964,30043.0,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,Victor Entertainment,"Funimation, Geneon Entertainment USA",Madhouse,Manga,24 min. per ep.,PG-13 - Teens 13 or older,266.0,201,558913,12944,29113,343492,25465,13925,146918,50229.0,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,"TV Tokyo, Bandai Visual, Dentsu, Victor Entert...","Funimation, Bandai Entertainment",Sunrise,Original,25 min. per ep.,PG-13 - Teens 13 or older,2481.0,1467,94683,587,4300,46165,5121,5378,33719,2182.0,4806.0,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,52,"Sep 30, 2004 to Sep 29, 2005",Fall 2004,"TV Tokyo, Dentsu",Unknown,Toei Animation,Manga,23 min. per ep.,PG - Children,3710.0,4369,13224,18,642,7314,766,1108,3394,312.0,529.0,1242.0,1713.0,1068.0,634.0,265.0,83.0,50.0,27.0


In [9]:
anime.shape

(17562, 35)

In [10]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17562 entries, 0 to 17561
Data columns (total 35 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MAL_ID         17562 non-null  int64 
 1   Name           17562 non-null  object
 2   Score          17562 non-null  object
 3   Genres         17562 non-null  object
 4   English name   17562 non-null  object
 5   Japanese name  17562 non-null  object
 6   Type           17562 non-null  object
 7   Episodes       17562 non-null  object
 8   Aired          17562 non-null  object
 9   Premiered      17562 non-null  object
 10  Producers      17562 non-null  object
 11  Licensors      17562 non-null  object
 12  Studios        17562 non-null  object
 13  Source         17562 non-null  object
 14  Duration       17562 non-null  object
 15  Rating         17562 non-null  object
 16  Ranked         17562 non-null  object
 17  Popularity     17562 non-null  int64 
 18  Members        17562 non-n

In [11]:
ratings = pd.read_csv('/home/dmitry-ds/rec-sys/data/rating_complete.csv')

In [12]:
ratings.head()

,user_id,anime_id,rating
0,0,430,9
1,0,1004,5
2,0,3010,7
3,0,570,7
4,0,2762,9


In [13]:
ratings.shape

(57633278, 3)

In [14]:
ratings['rating'].unique()

array([ 9,  5,  7,  8, 10,  6,  4,  3,  2,  1])

In [15]:
N_USER = 15000 #3000

In [16]:
rating_selected = ratings.head(np.where(ratings['user_id'] == N_USER)[0][-1]+1)

In [17]:
rating_selected.shape

(2435913, 3)

In [18]:
rating_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2435913 entries, 0 to 2435912
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 55.8 MB


## creating users df with features

In [19]:
anime.rename(columns = {'MAL_ID': 'anime_id'}, inplace = True)

In [20]:
rating_selected = rating_selected.merge(anime[['anime_id', 'Genres']], on = 'anime_id', how = 'left')

In [21]:
rating_selected.head()

,user_id,anime_id,rating,Genres
0,0,430,9,"Military, Comedy, Historical, Drama, Fantasy, ..."
1,0,1004,5,"Drama, Psychological, Romance, Slice of Life"
2,0,3010,7,"Adventure, Historical, Shounen"
3,0,570,7,"Military, Police, Psychological, Drama, Romance"
4,0,2762,9,"Action, Adventure, Comedy, Romance, School, Sh..."


In [22]:
total_genres = sorted(set([i.strip() for i in itertools.chain.from_iterable([i.split(',') for i in anime['Genres'].values])]))

In [23]:
total_genres[:5]

['Action', 'Adventure', 'Cars', 'Comedy', 'Dementia']

In [24]:
def create_users(rating_selected):

    user_data = []

    for user_id in tqdm(rating_selected['user_id'].unique()):
        filter_user = rating_selected[rating_selected['user_id'] == user_id]
        total_anime_watch = len(filter_user)
        genres_user = filter_user['Genres']

        if isinstance(genres_user, str):
            genres_user = [genres_user]
        else:
            genres_user = genres_user.values

        user_gen_dat = {}

        all_genres = [x.strip() for x in (','.join(map(lambda x: str(x), genres_user))).split(',')]

        for unique_gen in total_genres:
            user_gen_dat[unique_gen] = all_genres.count(unique_gen)/total_anime_watch
        
        user_gen_dat['mean_rating'] = filter_user['rating'].sum()/total_anime_watch
        
        user_data.append({user_id: user_gen_dat})
        
    users = pd.DataFrame(user_data).stack().apply(pd.Series).reset_index().rename(columns = {'level_1': 'user_id'}).drop('level_0', axis = 1)
    
    return users

In [25]:
users = create_users(rating_selected)

100%|██████████| 13102/13102 [00:40<00:00, 324.27it/s]


In [26]:
users.head()

,user_id,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,Hentai,Historical,Horror,Josei,Kids,Magic,Martial Arts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,Shoujo Ai,Shounen,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Unknown,Vampire,Yaoi,Yuri,mean_rating
0,0,0.285714,0.514286,0.000000,0.457143,0.000000,0.000000,0.428571,0.028571,0.314286,0.028571,0.000000,0.000000,0.171429,0.057143,0.000000,0.000000,0.114286,0.085714,0.000000,0.114286,0.028571,0.057143,0.028571,0.028571,0.057143,0.371429,0.028571,0.057143,0.114286,0.000000,0.085714,0.000000,0.342857,0.028571,0.114286,0.000000,0.028571,0.171429,0.200000,0.000000,0.0,0.057143,0.0,0.0,7.400000
1,1,0.427184,0.233010,0.000000,0.485437,0.000000,0.019417,0.368932,0.077670,0.223301,0.029126,0.009709,0.000000,0.029126,0.029126,0.000000,0.009709,0.038835,0.097087,0.029126,0.048544,0.009709,0.106796,0.038835,0.019417,0.087379,0.194175,0.000000,0.407767,0.135922,0.058252,0.000000,0.000000,0.524272,0.000000,0.087379,0.000000,0.174757,0.203883,0.213592,0.077670,0.0,0.009709,0.0,0.0,8.058252
2,2,0.568627,0.156863,0.000000,0.352941,0.000000,0.039216,0.294118,0.176471,0.078431,0.058824,0.117647,0.156863,0.000000,0.137255,0.000000,0.000000,0.019608,0.039216,0.058824,0.078431,0.019608,0.176471,0.039216,0.019608,0.196078,0.117647,0.000000,0.196078,0.196078,0.156863,0.000000,0.000000,0.450980,0.000000,0.039216,0.000000,0.078431,0.215686,0.313725,0.078431,0.0,0.039216,0.0,0.0,8.333333
3,3,0.377778,0.079365,0.025397,0.612698,0.009524,0.041270,0.219048,0.250794,0.152381,0.025397,0.209524,0.000000,0.015873,0.041270,0.003175,0.000000,0.069841,0.019048,0.041270,0.028571,0.006349,0.130159,0.050794,0.006349,0.076190,0.403175,0.009524,0.403175,0.146032,0.184127,0.022222,0.003175,0.196825,0.000000,0.212698,0.003175,0.025397,0.069841,0.295238,0.047619,0.0,0.031746,0.0,0.0,7.603175
4,4,0.271186,0.220339,0.000000,0.457627,0.000000,0.050847,0.584746,0.050847,0.330508,0.000000,0.050847,0.000000,0.110169,0.059322,0.050847,0.000000,0.194915,0.042373,0.093220,0.033898,0.042373,0.177966,0.016949,0.008475,0.059322,0.618644,0.000000,0.194915,0.127119,0.025424,0.347458,0.016949,0.177966,0.008475,0.237288,0.000000,0.008475,0.076271,0.288136,0.025424,0.0,0.033898,0.0,0.0,7.652542


In [27]:
users.shape

(13102, 46)

In [29]:
#users.to_csv('/home/dmitry-ds/rec-sys/data/user-preferences.csv', index = False)

## creating anime_selected df with features

In [30]:
anime['Genres_sep'] = anime['Genres'].apply(lambda x: [i.strip() for i in x.split(',')])

In [31]:
anime = anime.join(anime['Genres_sep'].str.join('|').str.get_dummies())

In [29]:
anime.head()

,anime_id,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,Producers,Licensors,Studios,Source,Duration,Rating,Ranked,Popularity,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1,Genres_sep,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,Hentai,Historical,Horror,Josei,Kids,Magic,Martial Arts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,Shoujo Ai,Shounen,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Unknown,Vampire,Yaoi,Yuri
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,Original,24 min. per ep.,R - 17+ (violence & profanity),28.0,39,1251960,61971,105808,718161,71513,26678,329800,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0,"[Action, Adventure, Comedy, Drama, Sci-Fi, Space]",1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,"Sunrise, Bandai Visual",Sony Pictures Entertainment,Bones,Original,1 hr. 55 min.,R - 17+ (violence & profanity),159.0,518,273145,1174,4143,208333,1935,770,57964,30043.0,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0,"[Action, Drama, Mystery, Sci-Fi, Space]",1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,Victor Entertainment,"Funimation, Geneon Entertainment USA",Madhouse,Manga,24 min. per ep.,PG-13 - Teens 13 or older,266.0,201,558913,12944,29113,343492,25465,13925,146918,50229.0,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0,"[Action, Sci-Fi, Adventure, Comedy, Drama, Sho...",1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,"TV Tokyo, Bandai Visual, Dentsu, Victor Entert...","Funimation, Bandai Entertainment",Sunrise,Original,25 min. per ep.,PG-13 - Teens 13 or older,2481.0,1467,94683,587,4300,46165,5121,5378,33719,2182.0,4806.0,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0,"[Action, Mystery, Police, Supernatural, Drama,...",1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,52,"Sep 30, 2004 to Sep 29, 2005",Fall 2004,"TV Tokyo, Dentsu",Unknown,Toei Animation,Manga,23 min. per ep.,PG - Children,3710.0,4369,13224,18,642,7314,766,1108,3394,312.0,529.0,1242.0,1713.0,1068.0,634.0,265.0,83.0,50.0,27.0,"[Adventure, Fantasy, Shounen, Supernatural]",0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0


In [30]:
anime['Rating'].nunique()

7

In [31]:
total_producers = sorted(set([i.strip() for i in itertools.chain.from_iterable([i.split(',') for i in anime['Producers'].values])]))

In [32]:
len(total_producers)

1307

In [33]:
total_licensors = sorted(set([i.strip() for i in itertools.chain.from_iterable([i.split(',') for i in anime['Licensors'].values])]))

In [34]:
len(total_licensors)

79

In [35]:
total_studios = sorted(set([i.strip() for i in itertools.chain.from_iterable([i.split(',') for i in anime['Studios'].values])]))

In [36]:
len(total_studios)

723

In [37]:
def parse_start_date(s):
    try:
        start_date = re.match(r'(\w*\s\d*,\s)(\d*)', s).group(2)
        if int(start_date[:3]) < 198:
            start_date = 'old'
        elif '198' in start_date:
            start_date = '80s'
        elif '199' in start_date:
            start_date = '90s'
        elif '200' in start_date:
            start_date = '00s'
        elif '201' in start_date:
            start_date = '10s'
        elif '202' in start_date:
            start_date = 'new'
    except:
        start_date = 'Unknown'
    return start_date

In [38]:
anime['Start air'] = anime['Aired'].apply(lambda x: parse_start_date(x))

In [39]:
anime['Start air'].value_counts()

10s        7448
00s        3634
Unknown    2109
90s        1865
80s        1109
new         770
old         627
Name: Start air, dtype: int64

In [40]:
anime_selected = anime[anime['anime_id'].isin(rating_selected['anime_id'].values)]

In [41]:
anime_selected.shape

(14353, 81)

## constructing dataset

In [42]:
item_cat_cols = ['Type', 'Source', 'Rating', 'Start air']

for col in item_cat_cols:
    try:
        anime_selected = pd.merge(anime_selected, anime_selected[col].str.get_dummies().drop('Unknown', axis = 1), left_index=True, right_index=True)
    except:
        anime_selected = pd.merge(anime_selected, anime_selected[col].str.get_dummies(), left_index=True, right_index=True)

In [43]:
anime_selected[['Score', 'Episodes', 'Ranked', 'Popularity',
                'Members', 'Favorites', 'Watching', 'Completed', 'On-Hold', 'Dropped',
                'Plan to Watch', 'Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6',
                'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1']] = anime_selected[['Score', 'Episodes', 'Ranked', 'Popularity',
                'Members', 'Favorites', 'Watching', 'Completed', 'On-Hold', 'Dropped',
                'Plan to Watch', 'Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6',
                'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1']].replace({'Unknown': '0'})

In [44]:
anime_selected['Score'] = anime_selected['Score'].astype('float64')
anime_selected[['Episodes', 'Ranked','Popularity','Members',
                'Favorites','Watching','Completed','On-Hold',
                'Dropped','Plan to Watch','Score-10','Score-9',
                'Score-8','Score-7','Score-6','Score-5',
                'Score-4','Score-3','Score-2','Score-1']] = anime_selected[['Episodes','Ranked','Popularity','Members',
                                              'Favorites','Watching','Completed','On-Hold',
                                              'Dropped','Plan to Watch','Score-10','Score-9',
                                              'Score-8','Score-7','Score-6','Score-5',
                                              'Score-4','Score-3','Score-2','Score-1']].astype('float64')

In [45]:
min_max_scaler = preprocessing.MinMaxScaler()
anime_selected[['Score', 'Episodes', 'Ranked', 'Popularity',
                'Members', 'Favorites', 'Watching', 'Completed', 'On-Hold', 'Dropped',
                'Plan to Watch', 'Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6',
                'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1']] = min_max_scaler.fit_transform(anime_selected[['Score', 'Episodes', 'Ranked', 'Popularity',
                'Members', 'Favorites', 'Watching', 'Completed', 'On-Hold', 'Dropped',
                'Plan to Watch', 'Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6',
                'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1']])

In [46]:
min_max_scaler = preprocessing.MinMaxScaler()
users[['mean_rating']] = min_max_scaler.fit_transform(users[['mean_rating']])

In [47]:
item_features_columns = ['Score', 'Episodes', 'Ranked', 'Popularity',
                        'Members', 'Favorites', 'Watching', 'Completed', 'On-Hold', 'Dropped',
                         'Plan to Watch', 'Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6',
                         'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1', 
                         'Action', 'Adventure', 'Cars', 'Comedy', 'Dementia', 'Demons', 'Drama',
                         'Ecchi', 'Fantasy', 'Game_x', 'Harem', 'Hentai', 'Historical', 'Horror',
                         'Josei', 'Kids', 'Magic', 'Martial Arts', 'Mecha', 'Military', 'Music_x',
                         'Mystery', 'Parody', 'Police', 'Psychological', 'Romance', 'Samurai',
                         'School', 'Sci-Fi', 'Seinen', 'Shoujo', 'Shoujo Ai', 'Shounen',
                         'Shounen Ai', 'Slice of Life', 'Space', 'Sports', 'Super Power',
                         'Supernatural', 'Thriller', 'Unknown', 'Vampire', 'Yaoi', 'Yuri', 
                         'Movie', 'Music_y', 'ONA', 'OVA', 'Special', 'TV', '4-koma manga',
                         'Book', 'Card game', 'Digital manga', 'Game_y', 'Light novel', 'Manga',
                         'Music', 'Novel', 'Original', 'Other', 'Picture book', 'Radio', 'Visual novel',
                         'Web manga', 'G - All Ages', 'PG - Children', 'PG-13 - Teens 13 or older', 
                         'R - 17+ (violence & profanity)', 'R+ - Mild Nudity', 'Rx - Hentai',
                         '00s', '10s', '80s', '90s', 'new', 'old']

user_features_columns = [col for col in users.columns[1:]]

In [48]:
print(len(item_features_columns))
print(len(user_features_columns))

98
45


In [49]:
def construct_user_data(users):

    user_data = []

    for user_id in users['user_id'].unique():
        filter_user = users[users['user_id'] == user_id]

        user_gen_dat = dict(zip(user_features_columns, filter_user[user_features_columns].values[0]))

        user_data.append([user_id, user_gen_dat])
    
    return user_data

In [50]:
user_data = construct_user_data(users)

In [51]:
user_data[:2]

[[0,
  {'Action': 0.2857142857142857,
   'Adventure': 0.5142857142857142,
   'Cars': 0.0,
   'Comedy': 0.45714285714285713,
   'Dementia': 0.0,
   'Demons': 0.0,
   'Drama': 0.42857142857142855,
   'Ecchi': 0.02857142857142857,
   'Fantasy': 0.3142857142857143,
   'Game': 0.02857142857142857,
   'Harem': 0.0,
   'Hentai': 0.0,
   'Historical': 0.17142857142857143,
   'Horror': 0.05714285714285714,
   'Josei': 0.0,
   'Kids': 0.0,
   'Magic': 0.11428571428571428,
   'Martial Arts': 0.08571428571428572,
   'Mecha': 0.0,
   'Military': 0.11428571428571428,
   'Music': 0.02857142857142857,
   'Mystery': 0.05714285714285714,
   'Parody': 0.02857142857142857,
   'Police': 0.02857142857142857,
   'Psychological': 0.05714285714285714,
   'Romance': 0.37142857142857144,
   'Samurai': 0.02857142857142857,
   'School': 0.05714285714285714,
   'Sci-Fi': 0.11428571428571428,
   'Seinen': 0.0,
   'Shoujo': 0.08571428571428572,
   'Shoujo Ai': 0.0,
   'Shounen': 0.34285714285714286,
   'Shounen Ai': 

In [52]:
def construct_item_data(anime_selected):

    item_data = []

    for item_id in anime_selected['anime_id'].unique():
        filter_anime = anime_selected[anime_selected['anime_id'] == item_id]

        item_gen_dat = dict(zip(item_features_columns, filter_anime[item_features_columns].values[0]))

        item_data.append([item_id, item_gen_dat])
        
    return item_data

In [53]:
item_data = construct_item_data(anime_selected)

In [54]:
item_data[:2]

[[1,
  {'Score': 0.9553862894450489,
   'Episodes': 0.008505070330389271,
   'Ranked': 0.0017743979721166032,
   'Popularity': 0.0021684546907098836,
   'Members': 0.48345748809620354,
   'Favorites': 0.3369564035364355,
   'Watching': 0.18686102511483665,
   'Completed': 0.32904117911450953,
   'On-Hold': 0.5460633318316139,
   'Dropped': 0.15269875794173202,
   'Plan to Watch': 0.7750284942130309,
   'Score-10': 0.32060222912070463,
   'Score-9': 0.3402621568905861,
   'Score-8': 0.2866941254113911,
   'Score-7': 0.2051590945746232,
   'Score-6': 0.1097908518237445,
   'Score-5': 0.07133529350499523,
   'Score-4': 0.03923356539954408,
   'Score-3': 0.03069857931408922,
   'Score-2': 0.029206574435378975,
   'Score-1': 0.046814814814814816,
   'Action': 1.0,
   'Adventure': 1.0,
   'Cars': 0.0,
   'Comedy': 1.0,
   'Dementia': 0.0,
   'Demons': 0.0,
   'Drama': 1.0,
   'Ecchi': 0.0,
   'Fantasy': 0.0,
   'Game_x': 0.0,
   'Harem': 0.0,
   'Hentai': 0.0,
   'Historical': 0.0,
   'Horro

In [55]:
dataset = Dataset()

In [56]:
dataset.fit(
    users['user_id'].unique(),
    anime_selected['anime_id'].unique(),
    user_features = user_features_columns,
    item_features = item_features_columns
)

In [57]:
user_features = dataset.build_user_features(user_data, normalize = False)
item_features = dataset.build_item_features(item_data, normalize = False)

In [58]:
user_features

<13102x13147 sparse matrix of type '<class 'numpy.float32'>'
	with 602692 stored elements in Compressed Sparse Row format>

In [59]:
item_features

<14353x14451 sparse matrix of type '<class 'numpy.float32'>'
	with 1420947 stored elements in Compressed Sparse Row format>

In [60]:
(interactions, weights) = dataset.build_interactions([(x[0], x[1], x[2]) for x in rating_selected.values])

## train test split

In [62]:
SEED = 42
TEST_PERCENTAGE = 0.2

In [63]:
train_interactions, test_interactions = cross_validation.random_train_test_split(interactions,
                                                                                 test_percentage = TEST_PERCENTAGE,
                                                                                 random_state = np.random.RandomState(SEED)
                                                                                )

In [65]:
train_weights, test_weights = cross_validation.random_train_test_split(weights,
                                                                       test_percentage = TEST_PERCENTAGE,
                                                                       random_state = np.random.RandomState(SEED)
                                                                      )

In [69]:
interactions

<13102x14353 sparse matrix of type '<class 'numpy.int32'>'
	with 2435913 stored elements in COOrdinate format>

In [67]:
train_interactions

<13102x14353 sparse matrix of type '<class 'numpy.float32'>'
	with 1948730 stored elements in COOrdinate format>

In [66]:
train_weights

<13102x14353 sparse matrix of type '<class 'numpy.float32'>'
	with 1948730 stored elements in COOrdinate format>

## model training with cross val and choosing epoch num

In [70]:
for n_epoch in [10, 30, 50, 100]:
    
    model = LightFM(no_components = 64, loss = "warp", k = 15, random_state = 77)
    
    model.fit(
        interactions = train_interactions,
        sample_weight = train_weights,
        item_features = item_features,
        user_features = user_features,
        verbose = True,
        epochs = n_epoch,
        num_threads = 20
    )
    
    print(f'Num epoch: {n_epoch}')
    
    test_auc = auc_score(model,
                      test_interactions,
                      item_features = item_features,
                      user_features = user_features
                     ).mean()
    
    test_precision_at_k = precision_at_k(model,
                      test_interactions,
                      item_features = item_features,
                      user_features = user_features
                     ).mean()
    
    test_recall_at_k = recall_at_k(model,
                      test_interactions,
                      item_features = item_features,
                      user_features = user_features
                     ).mean()
    
    print(f'Test AUC: {test_auc}, test precision@k: {test_precision_at_k}, test recall@k10: {test_recall_at_k}')

Epoch: 100%|██████████| 10/10 [21:19<00:00, 127.90s/it]


Num epoch: 10
Test AUC: 0.9742241501808167, test precision@k: 0.13526353240013123, test recall@k10: 0.09051859795908473


Epoch: 100%|██████████| 30/30 [58:38<00:00, 117.29s/it] 


Num epoch: 30
Test AUC: 0.9751186966896057, test precision@k: 0.13185636699199677, test recall@k10: 0.09053137199218161


Epoch: 100%|██████████| 50/50 [1:32:47<00:00, 111.34s/it]


Num epoch: 50
Test AUC: 0.9754911661148071, test precision@k: 0.13045670092105865, test recall@k10: 0.09071833029967415


Epoch: 100%|██████████| 100/100 [2:53:13<00:00, 103.93s/it] 


Num epoch: 100
Test AUC: 0.975334644317627, test precision@k: 0.12735344469547272, test recall@k10: 0.08892039498706805


## train best

In [71]:
model = LightFM(no_components = 64, loss = "warp", k = 15) #no_components = 30
model.fit(
    interactions = interactions,
    sample_weight = weights,
    item_features = item_features,
    user_features = user_features,
    verbose = True,
    epochs = 50,
    num_threads = 20)

Epoch: 100%|██████████| 50/50 [1:43:00<00:00, 123.62s/it]


In [72]:
model

In [73]:
train_auc = auc_score(model,
                      interactions,
                      item_features = item_features,
                      user_features = user_features
                     ).mean()

In [74]:
train_auc

0.99164087

In [75]:
train_precision_at_k = precision_at_k(model,
                      interactions,
                      item_features = item_features,
                      user_features = user_features
                     ).mean()

In [76]:
train_precision_at_k

0.77181345

In [77]:
train_recall_at_k = recall_at_k(model,
                      interactions,
                      item_features = item_features,
                      user_features = user_features
                     ).mean()

In [78]:
train_recall_at_k

0.13919626509101557

## inference

In [79]:
#mapping of user_id (key) to serial number (value)

dataset.mapping()[0]

{0: 0,
 1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 5,
 6: 6,
 7: 7,
 8: 8,
 10: 9,
 11: 10,
 12: 11,
 13: 12,
 14: 13,
 15: 14,
 16: 15,
 17: 16,
 18: 17,
 19: 18,
 20: 19,
 21: 20,
 22: 21,
 23: 22,
 24: 23,
 25: 24,
 27: 25,
 28: 26,
 29: 27,
 30: 28,
 31: 29,
 32: 30,
 33: 31,
 34: 32,
 35: 33,
 36: 34,
 37: 35,
 38: 36,
 40: 37,
 41: 38,
 42: 39,
 43: 40,
 44: 41,
 45: 42,
 46: 43,
 47: 44,
 48: 45,
 49: 46,
 50: 47,
 51: 48,
 52: 49,
 53: 50,
 54: 51,
 55: 52,
 56: 53,
 57: 54,
 58: 55,
 59: 56,
 61: 57,
 62: 58,
 63: 59,
 64: 60,
 65: 61,
 66: 62,
 67: 63,
 68: 64,
 70: 65,
 71: 66,
 72: 67,
 73: 68,
 74: 69,
 75: 70,
 77: 71,
 78: 72,
 79: 73,
 80: 74,
 81: 75,
 82: 76,
 83: 77,
 84: 78,
 85: 79,
 87: 80,
 88: 81,
 89: 82,
 90: 83,
 91: 84,
 92: 85,
 93: 86,
 94: 87,
 95: 88,
 97: 89,
 99: 90,
 101: 91,
 102: 92,
 103: 93,
 104: 94,
 105: 95,
 107: 96,
 108: 97,
 110: 98,
 111: 99,
 112: 100,
 113: 101,
 117: 102,
 118: 103,
 119: 104,
 120: 105,
 121: 106,
 122: 107,
 123: 108,
 124: 109,


In [80]:
#mapping of user_features (key) to serial number (value)

dataset.mapping()[1].keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 62, 63, 64, 65, 66, 67, 68, 70, 71, 72, 73, 74, 75, 77, 78, 79, 80, 81, 82, 83, 84, 85, 87, 88, 89, 90, 91, 92, 93, 94, 95, 97, 99, 101, 102, 103, 104, 105, 107, 108, 110, 111, 112, 113, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 133, 134, 135, 136, 137, 138, 139, 141, 143, 145, 146, 147, 148, 149, 150, 152, 153, 154, 155, 156, 157, 158, 160, 162, 163, 164, 165, 166, 167, 169, 170, 171, 172, 173, 174, 177, 178, 179, 181, 182, 184, 185, 186, 187, 188, 189, 190, 192, 193, 194, 195, 196, 197, 198, 199, 201, 202, 204, 205, 207, 208, 209, 210, 211, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 235, 237, 238, 239, 240, 241, 243, 244, 245, 246, 247, 248, 249, 251, 252, 253, 254, 255,

In [81]:
#mapping of anime_id (key) to serial number (value)

dataset.mapping()[2].keys()

dict_keys([1, 5, 6, 7, 8, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 71, 72, 73, 74, 75, 76, 77, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 236, 237, 238, 239, 24

In [82]:
#mapping of anime_features (key) to serial number (value)

dataset.mapping()[3].keys()

dict_keys([1, 5, 6, 7, 8, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 71, 72, 73, 74, 75, 76, 77, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 236, 237, 238, 239, 24

In [83]:
n_users, n_items = interactions.shape

In [124]:
# def recommendME(model, anime, dataset, user_id = None, new_user_feature = None, k = 5):
#     nanime = anime.set_index('anime_id')
#     if user_id is None:
#         user_id = n_users + 1
#     if user_id > n_users or user_id not in users['user_id'].unique(): #> n_users
#         user_id = n_users + 1
#         for genre in total_genres:
#             if genre not in list(new_user_feature.keys()):
#                 new_user_feature[genre] = 0
#         dataset.fit_partial(users = [user_id], user_features = total_genres)
#     # My Feature matrix
#     # new_user_feature = [user_id,{' Adventure': 0.11764705882352941, ' Cars': 0.0, ' Comedy': 0.23529411764705882, ' Dementia': 0.0, ' Demons': 0.058823529411764705, ' Drama': 0.17647058823529413, ' Ecchi': 0.058823529411764705, ' Fantasy': 0.35294117647058826, ' Game': 0.058823529411764705, ' Harem': 0.0, ' Hentai': 0.0, ' Historical': 0.0, ' Horror': 0.058823529411764705, ' Josei': 0.0, ' Kids': 0.0, ' Magic': 0.11764705882352941, ' Martial Arts': 0.0, ' Mecha': 0.058823529411764705, ' Military': 0.11764705882352941, ' Music': 0.0, ' Mystery': 0.058823529411764705, ' Parody': 0.058823529411764705, ' Police': 0.17647058823529413, ' Psychological': 0.17647058823529413, ' Romance': 0.23529411764705882, ' Samurai': 0.0, ' School': 0.29411764705882354, ' Sci-Fi': 0.17647058823529413, ' Seinen': 0.11764705882352941, ' Shoujo': 0.058823529411764705, ' Shoujo Ai': 0.0, ' Shounen': 0.29411764705882354, ' Shounen Ai': 0.0, ' Slice of Life': 0.17647058823529413, ' Space': 0.0, ' Sports': 0.058823529411764705, ' Super Power': 0.17647058823529413, ' Supernatural': 0.47058823529411764, ' Thriller': 0.17647058823529413, ' Vampire': 0.0, ' Yaoi': 0.0, ' Yuri': 0.0, 'Action': 0.47058823529411764, 'Adventure': 0.058823529411764705, 'Cars': 0.0, 'Comedy': 0.11764705882352941, 'Dementia': 0.0, 'Demons': 0.0, 'Drama': 0.23529411764705882, 'Ecchi': 0.0, 'Fantasy': 0.0, 'Game': 0.0, 'Harem': 0.0, 'Hentai': 0.0, 'Historical': 0.0, 'Horror': 0.0, 'Josei': 0.0, 'Kids': 0.0, 'Magic': 0.0, 'Martial Arts': 0.0, 'Mecha': 0.0, 'Military': 0.0, 'Music': 0.0, 'Mystery': 0.058823529411764705, 'Parody': 0.0, 'Police': 0.0, 'Psychological': 0.0, 'Romance': 0.0, 'Samurai': 0.0, 'School': 0.0, 'Sci-Fi': 0.058823529411764705, 'Seinen': 0.0, 'Shoujo': 0.0, 'Shounen': 0.0, 'Slice of Life': 0.0, 'Space': 0.0, 'Sports': 0.0, 'Super Power': 0.0, 'Supernatural': 0.0, 'Thriller': 0.0, 'Vampire': 0.0, 'Yaoi': 0.0} ] 
#         new_user_feature = [user_id, new_user_feature]
#         new_user_feature = dataset.build_user_features([new_user_feature], normalize = False)
#     user_id_map = dataset.mapping()[0][user_id] # just user_id -1
#     scores = model.predict(user_id_map, np.arange(n_items), user_features = new_user_feature)
#     rank = np.argsort(-scores)
#     selected_anime_id = np.array(list(dataset.mapping()[2].keys()))[rank]
#     top_items = nanime.loc[selected_anime_id]

#     return top_items['Name'][:k].values

In [84]:
def recommendME(model, anime, dataset, user_id = None, new_user_feature = None, k = 10):
    nanime = anime.set_index('anime_id')
    if user_id is None:
        user_id = n_users + 1
    if user_id > n_users or user_id not in users['user_id'].unique(): #> n_users
        user_id = n_users + 1
        for feature in user_features_columns:
            if feature not in list(new_user_feature.keys()):
                new_user_feature[feature] = 0
        dataset.fit_partial(users = [user_id], user_features = user_features_columns)

        new_user_feature = [user_id, new_user_feature]
        new_user_feature = dataset.build_user_features([new_user_feature], normalize = False)
    user_id_map = dataset.mapping()[0][user_id] # just user_id -1
    scores = model.predict(user_id_map, np.arange(n_items), user_features = new_user_feature)
    rank = np.argsort(-scores)
    
    selected_anime_id = np.array(list(dataset.mapping()[2].keys()))[rank]
    top_items = nanime.loc[selected_anime_id]

    return top_items['Name'][:k].values

In [90]:
user_r = rating_selected[rating_selected['user_id'] == 77] #77, 217, 1001

In [91]:
user_watched = user_r.merge(anime_selected, on = 'anime_id', how = 'left')[['Name', 'Genres_x', 'rating']]

In [92]:
user_watched

,Name,Genres_x,rating
0,Death Note,"Mystery, Police, Psychological, Supernatural, ...",10
1,Wolf's Rain,"Action, Adventure, Drama, Fantasy, Mystery, Sc...",9
2,Serial Experiments Lain,"Dementia, Drama, Mystery, Psychological, Sci-F...",10
3,Perfect Blue,"Dementia, Drama, Horror, Psychological",6
4,Paradise Kiss,"Comedy, Drama, Josei, Romance, Slice of Life",10
5,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",7
6,Nana,"Music, Slice of Life, Comedy, Drama, Romance, ...",10
7,Neon Genesis Evangelion,"Action, Sci-Fi, Dementia, Psychological, Drama...",7
8,Byousoku 5 Centimeter,"Drama, Romance, Slice of Life",9
9,Durarara!!,"Action, Mystery, Supernatural",7


In [94]:
recommendME(model, anime_selected, dataset, user_id = 77)

array(["Shinmai 'FUREAI' Net. Concept Movie", 'Animal 1',
       'Muteki-kyuu*Believer / Mirai Harmony',
       'Hakken Taiken Daisuki! Shimajirou', 'Gene Diver',
       'Sore Ike! Anpanman: Omusubiman', 'My Melody no Akazukin',
       'Manga Nihonshi (NHK Han)', 'Akai Koudan Zillion Recaps',
       'Unko-san: Tsuiteru Hito ni Shika Mienai Yousei Junjou Ha'],
      dtype=object)

In [89]:
recommendME(model, anime_selected, dataset, new_user_feature = {'Adventure': 0.5, 'Action': 0.2, 'Police': 0.3, 'Mystery': 0.8})

array(['Detective Conan Movie 14: The Lost Ship in the Sky',
       'Chi-Sui Maru', 'Detective Conan OVA 01: Conan vs. Kid vs. Yaiba',
       'Detective Conan Movie 15: Quarter of Silence',
       'Ton-Ton Atta to Niigata no Mukashibanashi',
       'Detective Conan Movie 16: The Eleventh Striker',
       'Detective Conan Movie 03: The Last Wizard of the Century',
       'Magic Kaito', 'Kakumeiteki Broadway Shugisha Doumei',
       'Detective Conan Movie 06: The Phantom of Baker Street'],
      dtype=object)

In [95]:
recommendME(model, anime_selected, dataset, user_id = 9, new_user_feature = {})

array(['Loups=Garous #Overture', 'Doutei Kawaiya', 'Minarai Chameleon',
       'Chi-Sui Maru', 'Fractale', 'Egao no Hana',
       'Sekiei Ayakashi Mangatan', 'Shoka', 'Jim Button',
       'Mahou Shoujo Sonico★Magica'], dtype=object)

## rec similar 

In [96]:
item_features

<14353x14451 sparse matrix of type '<class 'numpy.float32'>'
	with 1420947 stored elements in Compressed Sparse Row format>

In [97]:
_, item_representations = model.get_item_representations(features = item_features)

In [98]:
item_representations.shape

(14353, 64)

In [99]:
item_representations[0, :].shape

(64,)

In [100]:
def similar_items(anime_id, item_features, model, N = 10):

    _, item_representations = model.get_item_representations(features = item_features)
    
    item_id = dataset.mapping()[2][anime_id]
    
    # Cosine similarity
    scores = item_representations.dot(item_representations[item_id, :])
    item_norms = np.linalg.norm(item_representations, axis=1)
    item_norms[item_norms == 0] = 1e-10
    scores /= item_norms

    best = np.argpartition(scores, -(N + 1))[-(N + 1) :]
    return pd.DataFrame(
        sorted(zip(best, scores[best] / item_norms[item_id]), key = lambda x: -x[1])[1:],
        columns = ['itemID', 'score']
    )

In [105]:
anime_selected[anime_selected['Name'] == 'Neon Genesis Evangelion'] #Neon Genesis Evangelion

,anime_id,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,Producers,Licensors,Studios,Source,Duration,Rating,Ranked,Popularity,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1,Genres_sep,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game_x,Harem,Hentai,Historical,Horror,Josei,Kids,Magic,Martial Arts,Mecha,Military,Music_x,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,Shoujo Ai,Shounen,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Unknown,Vampire,Yaoi,Yuri,Start air,Movie,Music_y,ONA,OVA,Special,TV,4-koma manga,Book,Card game,Digital manga,Game_y,Light novel,Manga,Music,Novel,Original,Other,Picture book,Radio,Visual novel,Web manga,G - All Ages,PG - Children,PG-13 - Teens 13 or older,R - 17+ (violence & profanity),R+ - Mild Nudity,Rx - Hentai,00s,10s,80s,90s,new,old
20,30,Neon Genesis Evangelion,0.905332,"Action, Sci-Fi, Dementia, Psychological, Drama...",Neon Genesis Evangelion,新世紀エヴァンゲリオン,TV,0.008505,"Oct 4, 1995 to Mar 27, 1996",Fall 1995,"TV Tokyo, Kadokawa Shoten, Nihon Ad Systems, A...","ADV Films, Netflix","Gainax, Tatsunoko Production",Original,24 min. per ep.,PG-13 - Teens 13 or older,0.013181,0.003024,0.448196,0.387725,0.117528,0.37384,0.271577,0.116525,0.522258,0.261324,0.313975,0.343262,0.30786,0.203709,0.140772,0.119352,0.099719,0.128611,0.110044,"[Action, Sci-Fi, Dementia, Psychological, Dram...",1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,90s,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0


In [106]:
recs = similar_items(anime_id = 30, 
              item_features = item_features, 
              model = model)

In [107]:
recs

,itemID,score
0,21,0.944046
1,202,0.882294
2,313,0.872303
3,20,0.841801
4,1814,0.840228
5,2518,0.814544
6,0,0.809576
7,3234,0.804692
8,297,0.803140
9,23,0.782596


In [108]:
anime[anime['anime_id'].isin(recs['itemID'].values)]

,anime_id,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,Producers,Licensors,Studios,Source,Duration,Rating,Ranked,Popularity,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1,Genres_sep,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,Hentai,Historical,Horror,Josei,Kids,Magic,Martial Arts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,Shoujo Ai,Shounen,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Unknown,Vampire,Yaoi,Yuri,Start air
10,20,Naruto,7.91,"Action, Adventure, Comedy, Super Power, Martia...",Naruto,ナルト,TV,220,"Oct 3, 2002 to Feb 8, 2007",Fall 2002,"TV Tokyo, Aniplex, Shueisha",VIZ Media,Studio Pierrot,Manga,23 min. per ep.,PG-13 - Teens 13 or older,660.0,8,1830540,65586,137167,1462223,61734,99806,69610,216866.0,234481.0,345563.0,286175.0,108155.0,46886.0,15477.0,6098.0,3582.0,5310.0,"[Action, Adventure, Comedy, Super Power, Marti...",1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,00s
11,21,One Piece,8.52,"Action, Adventure, Comedy, Super Power, Drama,...",One Piece,ONE PIECE,TV,Unknown,"Oct 20, 1999 to ?",Fall 1999,"Fuji TV, TAP, Shueisha","Funimation, 4Kids Entertainment",Toei Animation,Manga,24 min.,PG-13 - Teens 13 or older,95.0,31,1352724,126645,887333,33,187919,136245,141194,292445.0,166186.0,141755.0,85424.0,35342.0,19019.0,8201.0,3675.0,2616.0,6340.0,"[Action, Adventure, Comedy, Super Power, Drama...",1,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,90s
13,23,Ring ni Kakero 1,6.38,"Action, Shounen, Sports",Unknown,リングにかけろ１,TV,12,"Oct 6, 2004 to Dec 15, 2004",Fall 2004,Unknown,Unknown,Toei Animation,Manga,25 min. per ep.,PG - Children,6493.0,7333,3648,16,170,1333,145,296,1704,69.0,61.0,152.0,260.0,236.0,143.0,88.0,40.0,32.0,28.0,"[Action, Shounen, Sports]",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,00s
179,202,Wolf's Rain,7.82,"Action, Adventure, Drama, Fantasy, Mystery, Sc...",Wolf's Rain,ウルフズレイン,TV,26,"Jan 7, 2003 to Jul 29, 2003",Winter 2003,"Bandai Visual, Asatsu DK, Fuji TV","Funimation, Bandai Entertainment",Bones,Original,23 min. per ep.,PG-13 - Teens 13 or older,803.0,528,271685,5716,12930,135944,14981,11858,95972,17941.0,21254.0,31099.0,25311.0,11350.0,5440.0,2206.0,736.0,424.0,296.0,"[Action, Adventure, Drama, Fantasy, Mystery, S...",1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,00s
273,297,Grenadier: Hohoemi no Senshi,6.72,"Action, Ecchi, Adventure, Comedy, Shounen",Grenadier:The Beautiful Warrior,グレネーダー 〜ほほえみの閃士〜,TV,12,"Oct 14, 2004 to Jan 13, 2005",Fall 2004,"Shochiku, Kadokawa Shoten, Toshiba Entertainment",Media Blasters,Group TAC,Manga,25 min. per ep.,PG-13 - Teens 13 or older,4815.0,2269,49526,69,1793,30372,1695,2365,13301,978.0,1762.0,4149.0,7034.0,4927.0,2742.0,1175.0,514.0,237.0,133.0,"[Action, Ecchi, Adventure, Comedy, Shounen]",1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,00s
289,313,Ace wo Nerae! (1979),6.83,"Sports, Drama, Romance, School, Shoujo",Aim for the Ace! (1979),エースをねらえ！,Movie,1,"Sep 8, 1979",Unknown,Unknown,Unknown,Madhouse,Unknown,1 hr. 30 min.,PG-13 - Teens 13 or older,4296.0,7835,2981,7,67,1674,53,93,1094,58.0,128.0,259.0,362.0,211.0,135.0,53.0,21.0,12.0,12.0,"[Sports, Drama, Romance, School, Shoujo]",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,old
2306,2518,Masuda Kousuke Gekijou Gag Manga Biyori 2,7.22,Comedy,Unknown,増田こうすけ劇場 ギャグマンガ日和2,TV,12,"Aug 5, 2006 to Oct 21, 2006",Summer 2006,"Bandai Visual, SKY Perfect Well Think, Kids St...",Unknown,Artland,Manga,5 min. per ep.,PG-13 - Teens 13 or older,2685.0,8032,2742,2,160,1427,231,135,789,120.0,164.0,268.0,315.0,151.0,81.0,30.0,19.0,7.0,6.0,[Comedy

# Create df with item representations

In [109]:
_, item_representations = model.get_item_representations(features = item_features)

In [110]:
item_representations.shape

(14353, 64)

In [111]:
item_representations[0, :].shape

(64,)

In [112]:
anime_selected.shape

(14353, 114)

In [113]:
lightfm_emb = pd.DataFrame(item_representations)

In [114]:
lightfm_emb 

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63
0,0.877783,0.216084,-0.503182,0.437487,0.054265,-0.824538,-0.622960,0.182473,-0.178679,-0.337158,0.521606,-0.549872,0.118328,-0.137754,0.410715,-0.180973,-0.570817,-0.862738,0.190253,0.391911,-1.206864,-0.376387,-0.502265,-0.220076,0.133199,0.218592,0.131855,0.518180,-0.287929,0.397931,-0.149840,-0.199122,-1.612970,-0.434727,-0.474510,1.269207,0.567623,-0.342300,-0.433264,0.279489,0.713963,0.378506,-0.007666,0.115169,-0.089850,-0.549659,-0.761420,-1.464966,-1.322069,-0.032957,-1.080959,-0.135092,-1.022020,-0.002554,-0.664350,1.142069,-0.472538,0.082486,-1.632876,-0.314429,-0.641308,0.009389,-0.651457,-0.240969
1,0.788837,-0.012186,-0.317833,0.314195,-0.047927,-0.258797,-0.414207,0.587654,0.290944,-0.492094,0.504216,-0.563243,0.316588,-0.221832,0.152622,-0.144501,-0.525736,-0.729610,0.249949,0.364802,-0.320432,-0.059109,-0.257882,-0.179054,0.331877,0.272099,0.241218,0.240626,-0.176915,0.388873,0.232570,0.054042,-0.909219,-0.864141,-0.413376,0.867266,0.163863,-0.675938,-0.288388,0.368793,0.560333,0.124126,-0.081069,-0.065063,-0.163645,-0.172389,0.004206,-0.851234,-1.313750,-0.196758,-0.470921,-0.179368,-0.767311,0.157949,-0.861186,0.856685,-0.251793,0.220200,-1.072088,-0.158940,-0.560269,0.128557,-0.165758,-0.639258
2,0.868641,0.247856,-0.517895,0.442624,0.064535,-0.810399,-0.361278,0.194449,0.122730,-0.322278,0.711811,-0.660453,0.076834,0.134945,0.468751,-0.225444,-0.431001,-0.651314,-0.162654,0.598493,-0.790972,-0.237844,-0.007045,0.141971,0.236932,0.732294,-0.201798,0.245262,-0.111632,0.341748,0.124955,-0.436397,-1.282260,-0.007148,-0.567485,1.175146,0.732044,0.149928,-0.422363,0.668099,0.613865,0.190118,0.362475,0.072596,-0.187930,-0.255868,-0.827494,-1.074873,-1.356851,0.251918,-0.932196,-0.277612,-0.571510,0.198246,-0.286605,0.608716,-0.230222,-0.109961,-1.065986,-0.419032,-0.337330,0.226134,-0.690568,-0.468791
3,0.546605,0.087473,-0.639145,0.283394,0.253443,-0.820313,0.015405,0.474382,0.302539,-0.800443,0.892969,-0.388261,0.277505,-0.042568,0.439123,-0.364895,-0.849158,-0.788575,0.695727,-0.200876,-0.219717,0.442171,-0.076308,0.130964,0.399064,0.634226,0.026199,0.226828,-0.177381,0.274950,-0.100955,-0.826547,-0.780638,-0.124821,0.035293,0.996021,0.628169,0.184121,-0.471690,0.642785,0.278115,0.660980,0.575491,0.364003,-0.893204,-0.185376,-1.167024,-0.783266,-1.177665,0.000044,-0.797066,0.242582,-0.447440,0.599809,-0.748350,0.321003,-0.302048,-0.031326,-1.207911,-0.126790,-0.859715,0.039976,-0.823537,-0.901160
4,0.959497,0.375417,-0.479217,-0.081518,-0.138866,-0.301535,-0.927677,0.765599,-0.184445,-0.320570,0.921715,-0.276462,0.420868,0.263651,0.025486,0.253380,0.012840,-0.516066,0.260879,0.372026,-0.636345,-0.337303,-0.005577,0.332600,-0.262290,0.593170,-0.763047,0.121525,0.793068,0.211355,0.182324,-1.226115,-0.244048,-0.026609,0.078271,0.030275,0.350184,0.056900,-0.562028,0.689275,-0.038999,0.420757,0.326340,0.389768,-0.718323,-0.335632,-0.879264,-0.076213,-0.404206,0.182044,-0.542010,-0.604022,-0.492324,0.067137,-0.451040,0.034096,-0.833235,-1.011305,0.126696,-0.055475,0.057071,0.505040,-0.150823,-0.898033
5,0.965165,0.327743,-0.364458,0.094789,-0.611873,-0.718518,-0.251417,0.442173,-0.149948,-0.292155,0.771636,-0.410992,0.130989,-0.139102,0.187671,-0.474504,-0.098513,-0.519818,0.459444,0.396408,-0.161549,-0.668509,-0.475768,0.469252,0.202544,0.498222,-0.516248,0.034024,0.438087,-0.017490,-0.026968,-0.153871,-0.715276,0.623084,-0.437557,0.437339,0.722798,0.764788,-0.375994,0.538735,0.294302,0.582181,0.397223,0.113380,0.090357,-0.539753,-1.003370,-0.813853,-1.020868,0.472357,0.045930,-0.718706,-1.052484,0.302925,0.016411,0.371506,-0.493003,-0.780127,-0.256646,0.110461,0.401323,0.527412,-0.128427,-1.454382
6,0.644794,0.456642,-0.320301,-0.109193,-0.465182,-0.376107,-0.570270,-0.048216,-0.036723,-0.185685,1.124023,-0.083316,0.290028,0.114563,0.

In [115]:
lightfm_emb_columns = ['lightfmemb'+str(i) for i in range(item_representations.shape[1])]

In [116]:
lightfm_emb.columns = lightfm_emb_columns

In [117]:
lightfm_emb.insert(loc = 0, column = 'anime_id', value = anime_selected['anime_id'].values)

In [118]:
lightfm_emb.head()

,anime_id,lightfmemb0,lightfmemb1,lightfmemb2,lightfmemb3,lightfmemb4,lightfmemb5,lightfmemb6,lightfmemb7,lightfmemb8,lightfmemb9,lightfmemb10,lightfmemb11,lightfmemb12,lightfmemb13,lightfmemb14,lightfmemb15,lightfmemb16,lightfmemb17,lightfmemb18,lightfmemb19,lightfmemb20,lightfmemb21,lightfmemb22,lightfmemb23,lightfmemb24,lightfmemb25,lightfmemb26,lightfmemb27,lightfmemb28,lightfmemb29,lightfmemb30,lightfmemb31,lightfmemb32,lightfmemb33,lightfmemb34,lightfmemb35,lightfmemb36,lightfmemb37,lightfmemb38,lightfmemb39,lightfmemb40,lightfmemb41,lightfmemb42,lightfmemb43,lightfmemb44,lightfmemb45,lightfmemb46,lightfmemb47,lightfmemb48,lightfmemb49,lightfmemb50,lightfmemb51,lightfmemb52,lightfmemb53,lightfmemb54,lightfmemb55,lightfmemb56,lightfmemb57,lightfmemb58,lightfmemb59,lightfmemb60,lightfmemb61,lightfmemb62,lightfmemb63
0,1,0.877783,0.216084,-0.503182,0.437487,0.054265,-0.824538,-0.622960,0.182473,-0.178679,-0.337158,0.521606,-0.549872,0.118328,-0.137754,0.410715,-0.180973,-0.570817,-0.862738,0.190253,0.391911,-1.206864,-0.376387,-0.502265,-0.220076,0.133199,0.218592,0.131855,0.518180,-0.287929,0.397931,-0.149840,-0.199122,-1.612970,-0.434727,-0.474510,1.269207,0.567623,-0.342300,-0.433264,0.279489,0.713963,0.378506,-0.007666,0.115169,-0.089850,-0.549659,-0.761420,-1.464966,-1.322069,-0.032957,-1.080959,-0.135092,-1.022020,-0.002554,-0.664350,1.142069,-0.472538,0.082486,-1.632876,-0.314429,-0.641308,0.009389,-0.651457,-0.240969
1,5,0.788837,-0.012186,-0.317833,0.314195,-0.047927,-0.258797,-0.414207,0.587654,0.290944,-0.492094,0.504216,-0.563243,0.316588,-0.221832,0.152622,-0.144501,-0.525736,-0.729610,0.249949,0.364802,-0.320432,-0.059109,-0.257882,-0.179054,0.331877,0.272099,0.241218,0.240626,-0.176915,0.388873,0.232570,0.054042,-0.909219,-0.864141,-0.413376,0.867266,0.163863,-0.675938,-0.288388,0.368793,0.560333,0.124126,-0.081069,-0.065063,-0.163645,-0.172389,0.004206,-0.851234,-1.313750,-0.196758,-0.470921,-0.179368,-0.767311,0.157949,-0.861186,0.856685,-0.251793,0.220200,-1.072088,-0.158940,-0.560269,0.128557,-0.165758,-0.639258
2,6,0.868641,0.247856,-0.517895,0.442624,0.064535,-0.810399,-0.361278,0.194449,0.122730,-0.322278,0.711811,-0.660453,0.076834,0.134945,0.468751,-0.225444,-0.431001,-0.651314,-0.162654,0.598493,-0.790972,-0.237844,-0.007045,0.141971,0.236932,0.732294,-0.201798,0.245262,-0.111632,0.341748,0.124955,-0.436397,-1.282260,-0.007148,-0.567485,1.175146,0.732044,0.149928,-0.422363,0.668099,0.613865,0.190118,0.362475,0.072596,-0.187930,-0.255868,-0.827494,-1.074873,-1.356851,0.251918,-0.932196,-0.277612,-0.571510,0.198246,-0.286605,0.608716,-0.230222,-0.109961,-1.065986,-0.419032,-0.337330,0.226134,-0.690568,-0.468791
3,7,0.546605,0.087473,-0.639145,0.283394,0.253443,-0.820313,0.015405,0.474382,0.302539,-0.800443,0.892969,-0.388261,0.277505,-0.042568,0.439123,-0.364895,-0.849158,-0.788575,0.695727,-0.200876,-0.219717,0.442171,-0.076308,0.130964,0.399064,0.634226,0.026199,0.226828,-0.177381,0.274950,-0.100955,-0.826547,-0.780638,-0.124821,0.035293,0.996021,0.628169,0.184121,-0.471690,0.642785,0.278115,0.660980,0.575491,0.364003,-0.893204,-0.185376,-1.167024,-0.783266,-1.177665,0.000044,-0.797066,0.242582,-0.447440,0.599809,-0.748350,0.321003,-0.302048,-0.031326,-1.207911,-0.126790,-0.859715,0.039976,-0.823537,-0.901160
4,8,0.959497,0.375417,-0.479217,-0.081518,-0.138866,-0.301535,-0.927677,0.765599,-0.184445,-0.320570,0.921715,-0.276462,0.420868,0.263651,0.025486,0.253380,0.012840,-0.516066,0.260879,0.372026,-0.636345,-0.337303,-0.005577,0.332600,-0.262290,0.593170,-0.763047,0.121525,0.793068,0.211355,0.182324,-1.226115,-0.244048,-0.026609,0.078271,0.030275,0.350184,0.056900,-0.562028,0.689275,-0.038999,0.420757,0.326340,0.389768,-0.718323,-0.335632,-0.879264,-0.076213,-0.404206,0.182044,-0.542010,-0.604022,-0.492324,0.067137,-0.451040,0.034096,-0.833235,-1.011305,0.126696,-0.055475,0.057071,0.505040,-0.150823,-0.898033


In [119]:
lightfm_emb.shape

(14353, 65)

In [120]:
#lightfm_emb.to_csv('/home/dmitry-ds/rec-sys/data/lightfm_emb_df_50e.csv', index = False)